In [1]:
# Env requirements
from dotenv import load_dotenv
import os
 
load_dotenv()
open_api_key = os.getenv('OPENAI_API_KEY')

In [2]:
# LangChain Imports
from langchain.llms import OpenAI

llm = OpenAI(
    openai_api_key=open_api_key,
    temperature=0.6
)

C:\Projects\LLM-Study\CrashCourse\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [3]:
user_input = "What's Israel Capital?"

print(llm.predict(user_input))

C:\Projects\LLM-Study\CrashCourse\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




The capital of Israel is Jerusalem.
